In [1]:
import numpy as np

In [2]:
# Function to train a decision tree
def DecisionTreeTrain(x, y, attributes, max_depth = 5, tree_depth = 0, multiplier = 1,
                      tree = None, leaf = 0, A = None, a = None, A_best = None, n = None):
    '''
    Input:
    x - Training examples. x is a m x n matrix, with n examples and m  features
    y - Labels, or target values. Currently only binary 0 or 1 (or -1 and 1) allowed. Must be a vector of size m
    attributes: List of numerical values to compare the training examples to. Currently, only a v x m array of 
                numerical values are accepted.
    max_depth: Default = 5. Maxmium depth of the tree
    multiplier: A constant to help better classify extreme cases. For the most common scenario, the most common is
                (multipler * positive cases) compared to (negative cases)
    
    Output:
    tree - Decision tree. It is formatted as an p x 5 array.
           Column 1 is the estimated value for the label given that branch and for x < a
           Column 2 is the feature (column) by which the data is split
           Column 3 is the value (a) by which the data is split. 
                    For the estimated label, that is the value given the feature (a - 1) < x < a, where 
                    a-1 is the next attribute below the listed (e.g., for a = -1.3 by the USDM classification,
                    then a-1 is the next attribute down the list (-1.6), so the label estimate is for -1.6 < x < -1.3)
           Column 4 is the current depth of the tree for that entry
           Column 5 is the current leaf for ths current branch
    
    '''

    # Initialize current node of the tree
    if tree_depth == 0:
        tree = np.zeros((1, 5))
        tree = np.append(tree, np.asarray([[None, None, None, tree_depth, leaf],]), axis = 0)
        tree = np.delete(tree, 0, axis = 0) # Delete the row of zeros
        n = tree.shape[0] - 1
    else:
        #tree[tree_depth+leaf] = [None, A_best, A[a], tree_depth, a]
        tree[n] = [None, A_best, A[a], tree_depth, a]
    
    # Get the total size of the data
    S_norm = y.size - np.sum(np.isnan(y))

    # Determine if the root node is pure
    if len(y[y == 1]) == S_norm: # All examples are positive
        #tree[tree_depth+leaf] = [1, A_best, A[a], tree_depth, a]
        tree[n] = [1, A_best, A[a], tree_depth, a]
        return tree
    elif len(y[y == -1]) == S_norm: # All examples are negative
        #tree[tree_depth+leaf] = [-1, A_best, A[a], tree_depth, a]
        tree[n] = [-1, A_best, A[a], tree_depth, a]
        return tree

    # If the attribute list is empty, or the tree is at its max depth, return the stump with the most common value
    if (np.sum(~np.isnan(attributes)) < 1) | (tree_depth >= max_depth): 
        most_common = 1 if (multiplier*np.sum(y == 1)) > np.sum(y == -1) else -1
        #tree[tree_depth+leaf] = [most_common, A_best, A[a], tree_depth, a]
        tree[n] = [most_common, A_best, A[a], tree_depth, a] 
        return tree
    
    V, M = attributes.shape
    
    # Following if block makes it so that attributes can be reused. Seemed needlessly complicated for 2 attributes, so it was left out for now.
    #if tree_depth == 0:
    #    A_best = BestAttribute(x, y, attributes)
    #    A = attributes[:,A_best]
    #else: # For none root branches, do not repeat the attribute just used.
    #    A_new = np.zeros((V, M))
    #    A_new[:,:] = attributes
    #    A_new[:,A_best] = np.nan
    #    
    #    A_best = BestAttribute(x, y, A_new)
    #    A = attributes[:,A_best]
    
    A_best = BestAttribute(x, y, attributes)
    A = attributes[:,A_best]
    
    A_new = np.zeros((V, M))
    A_new[:,:] = attributes
    A_new[:,A_best] = np.nan
    
    # Loop over all values for the best attribute
    for a in range(len(A)):
        # Split the data
        if (a == 0):
            xa = x[x[:,A_best] < A[a]]
            ya = y[x[:,A_best] < A[a]]
            leaf = 0 # If starting at a new set of attributes, reset the leaf
        else:
            xa = x[(x[:,A_best] >= A[a-1]) & (x[:,A_best] < A[a])]
            ya = y[(x[:,A_best] >= A[a-1]) & (x[:,A_best] < A[a])]
            leaf = a # Set the current leaf to whatever attribute index is being worked on
            
        # Add a node below the tree
        tree = np.append(tree, np.asarray([[None, A_best, A[a], tree_depth+1, a],]), axis = 0)
        n = tree.shape[0] - 1
        
        # If the examples are empty, assign the most common value
        if ya.size == 0:
            Sv_norm = 0
        else: # Note, theoretically, the Sv_norm < 1 condition is unneeded. But calculating it here helps accout for some sea grid points that may be nan
            Sv_norm = ya.size - np.sum(np.isnan(ya))

        if Sv_norm < 1:
            most_common = 1 if (multiplier*np.sum(ya == 1)) > np.sum(ya == -1) else -1
            #tree[tree_depth+leaf+1] = [most_common, A_best, A[a], tree_depth+1, a]
            tree[n] = [most_common, A_best, A[a], tree_depth+1, a]
            
        # If the subset is not empty, grow the tree further with that subset.
        else:
            tree = DecisionTreeTrain(xa, ya, A_new, max_depth = max_depth, multiplier = multiplier, tree_depth = tree_depth + 1, 
                                     tree = tree, leaf = a, A = A, a = a, A_best = A_best, n = n)
            #tree = DecisionTreeTrain(xa, ya, attributes, max_depth = max_depth, tree_depth = tree_depth + 1, 
            #                         tree = tree, leaf = a, A = A, a = a, A_best = A_best, n = n)
    
    return tree
        


# Function to determine the best attribute to split the data
def BestAttribute(x, y, attributes):
    '''
    Input:
    x - Training examples. x is a m x n matrix, with n examples and m  features
    y - Labels, or target values. Currently only binary 0 or 1 (or -1 and 1) allowed. Must be a vector of size m
    attributes: List of numerical values to compare the training examples to. Currently, only a v x m array of numerical values are accepted.
    max_depth: Default = 5. Maxmium depth of the tree
    
    Output:
    A - Best attribute to split the data
    '''
    
    
    M, N = x.shape
    V, N = attributes.shape
    
    # Get the total size
    if y.size == 0:
        S_norm = 0
    else:
        S_norm = y.size - np.sum(np.isnan(y))
    
    # Get the probability of positive and negative labels
    P_pos = np.sum(y == 1)/S_norm
    P_neg = np.sum(y == -1)/S_norm

    # Initialize the gain to the entropy
    gain = np.zeros((N))
    
    gain[:] = -1 * P_pos * np.log2(P_pos + 1e-5) - P_neg * np.log2(P_neg + 1e-5)
    
    for n in range(N):
        if np.sum(~np.isnan(attributes[:,n])) < 1:
            gain[n] = np.nan
            continue
            
        for v in range(V):
            # Split the data
            if (v == 0):
                Sv = y[x[:,n] < attributes[v,n]]
            else:
                Sv = y[(x[:,n] >= attributes[v-1,n]) & (x[:,n] < attributes[v,n])]
            
            # Get the size of each split
            if Sv.size == 0: # If the split dataset is empty, it does not contribute to the gain (SV_norm = 0)
                gain[n] = gain[n] 
            else:
                Sv_norm = Sv.size - np.sum(np.isnan(Sv))

                # Calculate probabilities
                Pv_pos = np.sum(Sv == 1)/Sv_norm
                Pv_neg = np.sum(Sv == -1)/Sv_norm

                # Calculate the entropy after each split
                Ev = -1 * Pv_pos * np.log2(Pv_pos + 1e-5) - Pv_neg * np.log2(Pv_neg + 1e-5)

                # Calculate the gain for this attribute
                gain[n] = gain[n] - Sv_norm * Ev/S_norm
        
    
    # Calculate the best attribute
    A = np.where(gain == np.nanmax(gain))[0][0]
    #A = attributes[gain == np.max(gain)]
    
    return A

    
# Function to make predictions with a decision tree

